In [1]:
import tushare as ts
import pandas as pd
import numpy as np
import os
import time
from tqdm import tqdm
import talib as ta
import mplfinance as mpf
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
import shutil
from tabulate import tabulate
import math

#pd.set_option("display.float_format", "{:.2f}".format)  # 显示两位小数
pd.set_option("display.colheader_justify", "left")  # 列头左对齐

TOEKN_PATH = os.path.expanduser("./.tushare.token")

with open(TOEKN_PATH, "r") as f:
    token = f.read().strip()
    ts.set_token(token=token)
    pro = ts.pro_api(token=token)


In [2]:
pro_stock_basic_df = pd.read_parquet('./pro_stock_basic.parquet')
pro_daily_df = pd.read_parquet('./pro_daily.parquet')
pro_daily_basic_df = pd.read_parquet('./pro_daily_basic.parquet')
pro_daily_indicator_df = pd.read_parquet('./pro_daily_indicator.parquet')
etf_df = pd.read_csv('etf/etf.csv')

def chnname(code):
    name = ""
    if code in pro_stock_basic_df["ts_code"].values:
        name = pro_stock_basic_df.loc[pro_stock_basic_df["ts_code"] == code, "name"].values[0]
    elif code in etf_df["ts_code"].values:
        name = etf_df.loc[etf_df["ts_code"] == code, "name"].values[0]
    return name


def apply_score(df, period=25):
    df.sort_values(by="trade_date", ascending=True, inplace=True)
    df = df.tail(period)
    y = np.log(df.close)
    x = np.arange(len(y))
    slope, intercept = np.polyfit(x, y, 1)
    annualized_returns = math.pow(math.exp(slope), 250) - 1
    r_squared = 1 - (
        sum((y - (slope * x + intercept)) ** 2) / ((len(y) - 1) * np.var(y, ddof=1))
    )
    score = annualized_returns * r_squared
    return score, annualized_returns, r_squared


def calc_net(df, single_pos):
    ATR_period = 14
    min_bid = 500
    min_grid = 0.005
    ATR_N = 5
    min_grid_min_profit = 30

    df.sort_values(by="trade_date", ascending=True, inplace=True)
    df["atr"] = ta.NATR(df["high"], df["low"], df["close"], timeperiod=ATR_period)
    atr = df["atr"].iloc[-1]
    close = df["close"].iloc[-1]

    bid_num = math.ceil(min_bid / close / 100) * 100
    # bid
    bid_price = bid_num * close
    bid_percent = bid_num * close * (ATR_N * atr / 100) / single_pos
    bid_profit = bid_price * bid_percent

    # check grid
    bid_grid = close * bid_percent
    if bid_grid < min_grid:
        bid_grid = min_grid
        bid_percent = bid_grid / close
        bid_price = bid_percent * single_pos / (ATR_N * atr / 100)
        bid_num = math.ceil(bid_price / close / 100) * 100
        bid_price = bid_num * close
        bid_percent = bid_num * close * (ATR_N * atr / 100) / single_pos
        bid_grid = round(close * bid_percent, 3)
        bid_profit = bid_price * bid_percent

    # check profit
    if bid_profit < min_grid_min_profit:
        bid_profit = min_grid_min_profit
        bid_price = math.pow((bid_profit * single_pos / (ATR_N * atr / 100)), 0.5)
        bid_percent = bid_price * (ATR_N * atr / 100) / single_pos
        bid_num = math.ceil(bid_price / close / 100) * 100
        bid_price = bid_num * close
        bid_percent = bid_num * close * (ATR_N * atr / 100) / single_pos
        bid_grid = round(close * bid_percent, 3)
        bid_profit = bid_price * bid_percent

    bid_grid = round(bid_grid, 3)
    bid_percent = round(bid_percent * 100, 1)
    atr = round(atr, 2)
    return bid_num, bid_grid, bid_price, bid_percent, bid_profit, atr, close


In [3]:
hold_dict = {"ts_code": [], "num": [], "price": []}


def append_hold(ts_code, num, price):
    hold_dict["ts_code"].append(ts_code)
    hold_dict["num"].append(num)
    hold_dict["price"].append(price)


append_hold("159307.SZ", 15200, 1.033)  # 100红利
append_hold("159338.SZ", 8300, 1.479)  # 中证 A500
append_hold("159501.SZ", 60800, 1.479)  # 纳指100
append_hold("159502.SZ", 7500, 0.996)  # 标普生物科技
append_hold("159509.SZ", 4700, 2.005)  # 纳指科技
append_hold("159518.SZ", 6900, 1.098)  # 标普油气
append_hold("159545.SZ", 12900, 1.156)  # 恒生红利低波
append_hold("159593.SZ", 6900, 1.086)  # A50
append_hold("159605.SZ", 17900, 0.836)  # 中概互联
append_hold("159758.SZ", 17100, 0.876)  # 红利50
append_hold("159915.SZ", 3800, 1.949)  # 创业板
append_hold("159920.SZ", 12700, 1.172)  # 恒生
append_hold("159980.SZ", 4500, 1.66)  # 有色
append_hold("159981.SZ", 5100, 1.478)  # 能源化工
append_hold("159985.SZ", 4100, 1.843)  # 豆粕
append_hold("161226.SZ", 8100, 0.916)  #  白银
append_hold("162411.SZ", 18700, 0.801)  # 华宝油气
append_hold("501018.SH", 5700, 1.015)  # 南方原油
append_hold("501300.SH", 7800, 0.953)  # 美元债
append_hold("510170.SH", 8700, 0.858)  # 商品
append_hold("510300.SH", 2000, 3.802)  # 沪深 300
append_hold("510500.SH", 1400, 5.459)  # 中证 500
append_hold("511020.SH", 200, 118.117)  # 活跃国债
append_hold("511090.SH", 200, 125.986)  # 30年国债
append_hold("511520.SH", 200, 112.58)  # 政金债
append_hold("512950.SH", 11600, 1.286)  # 央企改革
append_hold("513080.SH", 4900, 1.530)  # 法国 ETF
append_hold("513130.SH", 26500, 0.565)  # 恒生科技
append_hold("513290.SH", 6500, 1.16)  # 纳指生物科技
append_hold("513310.SH", 5200, 1.488)  # 中韩半导体
append_hold("513400.SH", 13100, 1.144)  # 道琼斯
append_hold("513630.SH", 12400, 1.299)  # 香港红利
append_hold("513650.SH", 37800, 1.586)  # 标普500
append_hold("513690.SH", 17500, 0.853)  # 恒生股息
append_hold("513730.SH", 5600, 1.345)  # 东南亚科技
append_hold("513800.SH", 5500, 1.368)  # 东证
append_hold("513850.SH", 5300, 1.843)  # 美国50
append_hold("513880.SH", 5700, 1.318)  #  日经 225
append_hold("515180.SH", 11600, 1.293)  # 100 红利
append_hold("516310.SH", 13100, 1.160)  # 银行
append_hold("518880.SH", 12200, 6.137)  # 黄金
append_hold("563020.SH", 13700, 1.147)  # 低波红利
append_hold("588000.SH", 7400, 1.002)  # 科创 50
append_hold("600036.SH", 300, 38.84)  #  招商银行
append_hold("600919.SH", 1100, 9.44)  #  江苏银行
append_hold("600926.SH", 700, 14.232)  # 杭州银行
append_hold("601009.SH", 1000, 10.41)  #  南京银行
append_hold("601838.SH", 600, 16.80)  #   成都银行


hold_df = pd.DataFrame(hold_dict)
hold_df.set_index("ts_code", inplace=True)


def apply_info(df):
    df["state"] = ""
    for ts_code in df.index:
        if ts_code in hold_df.index:
            df.loc[ts_code, "state"] = "hold"
            df.loc[ts_code, "num"] = hold_df.loc[ts_code, "num"]
            if "close" in df.columns:
                df.loc[ts_code, "per"] = round(
                    (df.loc[ts_code, "close"] - hold_df.loc[ts_code, "price"])
                    / hold_df.loc[ts_code, "price"]
                    * 100,
                    1,
                )

In [ ]:
pb_df = pro_stock_basic_df.set_index("ts_code")[[ "name", "industry"]]
apply_info(pb_df)
ind1 = pro_daily_basic_df.set_index("ts_code")[["trade_date","close","pe_ttm","pb","dv_ttm","circ_mv"]]
pb_df = pb_df.join(ind1)
ind2 = pro_daily_indicator_df.set_index("ts_code")[["ma_percent","ma60"]]
pb_df = pb_df.join(ind2)

pb_df["circ_mv"] = pb_df["circ_mv"] / 10000
pb_df["rank"] = pb_df["circ_mv"].rank(method="average", ascending=False)
pb_df["rank_mv"] = round(pb_df["rank"] / len(pb_df) * 100, 2)
pb_df["rank"] = pb_df["pb"].rank(method="average", ascending=False)
pb_df["rank_pb"] = round(pb_df["rank"] / len(pb_df) * 100, 2)
pb_df["rank_mvpb"] = np.maximum(pb_df["rank_mv"], pb_df["rank_pb"])

pb_df["roe"] = pb_df["pb"] / pb_df["pe_ttm"]
pb_df["rank"] = pb_df["roe"].rank(method="average", ascending=True)
pb_df["rank_roe"] = round(pb_df["rank"] / len(pb_df) * 100, 2)

pb_df["rank"] = pb_df["dv_ttm"].rank(method="average", ascending=True)
pb_df["rank_dv"] = round(pb_df["rank"] / len(pb_df) * 100, 2)

pb_df["rank"] = pb_df["ma_percent"].rank(method="average", ascending=True)
pb_df["rank_pro"] = round(pb_df["rank"] / len(pb_df) * 100, 2)

pb_df["c2ma60"] = pb_df["close"] / pb_df["ma60"]
pb_df["rank"] = pb_df["c2ma60"].rank(method="average", ascending=False)
pb_df["rank_pri"] = round(pb_df["rank"] / len(pb_df) * 100, 2)


pb_df["rank"] = (
    pb_df["rank_mvpb"] + pb_df["rank_dv"] + pb_df["rank_roe"] + pb_df["rank_pro"] + pb_df["rank_pri"]
) / 5

pb_df = pb_df.sort_values(by="rank", ascending=False)


print(
    tabulate(
        pb_df.head(50),
        headers="keys",
    )
)



In [ ]:
data = {"name": [], "ts_code": [], "weight": []}


def append_data(name, ts_code, weight):
    data["name"].append(name)
    data["ts_code"].append(ts_code)
    data["weight"].append(weight)


# 40% US
append_data("纳指100", "159501.SZ", 10)
append_data("标普500", "513650.SH", 10)
append_data("道琼斯", "513400.SH", 5)
append_data("美国50", "513850.SH", 1)
append_data("纳指科技", "159509.SZ", 1)
append_data("纳指生物科技", "513290.SH", 1)
append_data("标普生物科技", "159502.SZ", 1)
#append_data("标普消费", "159529.SZ", 1) ##
append_data("标普油气", "159518.SZ", 1)
append_data("法国", "513080.SH", 1)
#append_data("德国", "159561.SZ", 1) ##
#append_data("沙特", "520830.SH", 1) ##
append_data("日经", "513880.SH", 1)
append_data("东证", "513800.SH", 1)
append_data("中韩半导体", "513310.SH", 1)
append_data("东南亚科技", "513730.SH", 1)
#append_data("亚太精选", "159687.SZ", 1) ##
#append_data("新兴亚洲", "520580.SH", 1) ##

# # 25%
append_data("美债", "501300.SH", 1)
append_data("油气", "162411.SZ", 2)
append_data("豆粕", "159985.SZ", 2)
append_data("商品", "510170.SH", 2)
append_data("原油", "501018.SH", 1)
append_data("有色", "159980.SZ", 1)
append_data("能源化工", "159981.SZ", 1)
append_data("白银", "161226.SZ", 1)

append_data("黄金", "518880.SH", 8)

append_data("政金债", "511520.SH", 2)
append_data("30年国债", "511090.SH", 2)
append_data("活跃国债", "511020.SH", 2)

# # 35%
append_data("恒生", "159920.SZ", 2)
append_data("恒科", "513130.SH", 2)
append_data("中概互联", "159605.SZ", 2)
append_data("A500", "159338.SZ", 2)
append_data("中证A50", "159593.SZ", 2)
append_data("沪深300", "510300.SH", 2)
append_data("中证500", "510500.SH", 1)
append_data("中证1000", "512100.SH", 1)
append_data("中证2000", "159531.SZ", 1)
append_data("创业板", "159915.SZ", 1)
append_data("科创板", "588000.SH", 1)

append_data("银行", "516310.SH", 2)
append_data("中证红利", "515180.SH", 2)
append_data("红利低波", "563020.SH", 2)
append_data("红利低波100", "159307.SZ", 2)
append_data("红利质量", "159758.SZ", 2)
append_data("结构调整", "512950.SH", 2)
append_data("恒生红利低波", "159545.SZ", 2)
append_data("港股红利", "513630.SH", 2)
append_data("恒生高股息", "513690.SH", 2)

m = 750000
df = pd.DataFrame(data)


df.set_index("ts_code", inplace=True)
apply_info(df)

ind2 = pro_daily_indicator_df.set_index("ts_code")[["ma_percent"]]
df = df.join(ind2)

def piecewise_linear_function(x):
    if x < 0:
        return 0.5
    elif x > 1:
        return 2
    elif x <= 0.5:
        k1 = (1 - 0.5) / (0.5 - 0)
        return 0.5 + k1 * x
    elif 0.5 < x:
        k2 = (2 - 1) / (1 - 0.5)
        return 1 + k2 * (x - 0.5)

df["k"] = df["ma_percent"].apply(piecewise_linear_function)
df["k_weight"] = df["weight"] * df["k"]
df["weight"] = df["k_weight"] / df["k_weight"].sum() * 90
df["value"] = round(df["weight"] * 750000 * 0.01 / 100, 0) * 100
print(df["weight"].sum())
df = df.sort_values(by="weight", ascending=False)
sum = round(df["value"].sum(), 2)
res = m - sum
print(df["value"].sum(),df["value"].sum()/m)

print(
    tabulate(
        df,
        headers="keys",
    )
)


In [10]:

hold_df = pd.DataFrame(hold_dict)
hold_df.set_index("ts_code", inplace=True)
hold_df.loc[:, "value"] = 10000  
hold_df.loc[hold_df.index.intersection(df.index), "value"] = df.loc[hold_df.index.intersection(df.index), "value"]
ind2 = pro_daily_indicator_df.set_index("ts_code")[["ma_percent"]]
hold_df = hold_df.join(ind2)
hold_df["ma_percent"] = round(hold_df["ma_percent"]*100,2)
hold_df = hold_df.join(pb_df["rank"])


for i in hold_df.index:
    history_df = pro_daily_df[pro_daily_df["ts_code"] == i]
    bid_num, bid_grid, bid_price, bid_percent, bid_profit, atr, close = calc_net(
        history_df, 10000
    )
    hold_df.loc[i, "bid_num"] = bid_num
    hold_df.loc[i, "bid_grid"] = bid_grid
    hold_df.loc[i, "bid_price"] = bid_price
    hold_df.loc[i, "bid_percent"] = bid_percent
    hold_df.loc[i, "bid_profit"] = bid_profit
    hold_df.loc[i, "atr"] = atr
    hold_df.loc[i, "close"] = close


print(
    tabulate(
        hold_df,
        headers="keys",
    )
)




,num,price,value,ma_percent,rank
ts_code,,,,,
159307.SZ,15200,1.033,9600,27.73,NaN
159338.SZ,8300,1.479,7100,7.39,NaN
159501.SZ,60800,1.479,89500,72.27,NaN
159502.SZ,7500,0.996,3100,0.54,NaN
159509.SZ,4700,2.005,9900,79.83,NaN
159518.SZ,6900,1.098,12300,99.16,NaN
159545.SZ,12900,1.156,11100,39.50,NaN
159593.SZ,6900,1.086,9800,28.91,NaN
159605.SZ,17900,0.836,7300,8.74,NaN
